https://www.infiniteloop.co.jp/blog/2018/01/learning-keras-05/  
2018年01月23日 (火) 著者 ： nobuh 

# training.py

In [1]:
# model/training.py

from keras.models import Sequential,load_model
from keras.layers import Dense,Activation
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

np.random.seed(0) # 乱数を固定値で初期化し再現性を持たせる

"""
データ準備
"""

iris=datasets.load_iris()
X=iris.data
T=iris.target

T=np_utils.to_categorical(T) # 数値を、位置に変換 [0,1,2]==>[[1,0,0],[0,1,0],[0,0,1]]
train_x,test_x,train_t,test_t=train_test_split(X,T,train_size=0.8,test_size=0.2) # 訓練とテストで分割

"""
モデル作成
"""

model=Sequential()
model.add(Dense(input_dim=4,units=3))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy",optimizer=SGD(lr=0.1))

"""
トレーニング
"""

# 損失関数がより小さくなったときだけモデルをファイル保存するコールバック
checkpointer=ModelCheckpoint(filepath="iris.h5",save_best_only=True)

# 学習結果をモニターしながらfitさせるために、テストデータとその正解をvalidation_dataとして付加
model.fit(train_x,train_t,epochs=40,batch_size=10,validation_data=(test_x,test_t),callbacks=[checkpointer])

"""
新たに作ったモデルにファイルから読み込む
"""
app=load_model("iris.h5")

"""
ファイルから作ったモデルで分類する
"""
Y=app.predict_classes(test_x,batch_size=10)

"""
結果検証
"""

_,T_index=np.where(test_t>0) # to_categoricalの逆変換
print()
print("RESULT")
print(Y==T_index)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 120 samples, validate on 30 samples
Epoch 1/40
120/120 [==============================] - 1s 6ms/step - loss: 1.7483 - val_loss: 1.2433
Epoch 2/40
120/120 [==============================] - 0s 276us/step - loss: 0.8176 - val_loss: 0.8857
Epoch 3/40
120/120 [==============================] - 0s 277us/step - loss: 0.9922 - val_loss: 0.7674
Epoch 4/40
120/120 [==============================] - 0s 260us/step - loss: 0.7833 - val_loss: 0.5768
Epoch 5/40
120/120 [==============================] - 0s 277us/step - loss: 0.5650 - val_loss: 0.4165
Epoch 6/40
120/120 [==============================] - 0s 240us/step - loss: 0.4934 - val_loss: 0.6005
Epoch 7/40
120/120 [==============================] - 0s 242us/step - loss: 0.8877 - val_loss: 1.0208
Epoch 8/40
120/120 [==============================] - 0s 232us/step - loss: 0.7910 - val_loss: 1.9515
Epoch 9/40
120/120 [=

# app.py

In [ ]:
# docroot/app.py

from flask import Flask,request
import numpy as np
import keras.models
import tensorflow as tf

app=Flask(__name__)

# model and backend graph must be created on global
global model,graph
model=keras.models.load_model("../model/iris.h5")
graph=tf.get_default_graph()

@app.route("/",methods=["GET"])
def root():
    names=[
        'Iris-Setosa セトナ ヒオウギアヤメ',
        'Iris-Versicolour バーシクル ブルーフラッグ',
        'Iris-Virginica バージニカ'
    ]
    
    sl=sw=pl=pw=0.0
    sl=request.args.get("sepal_l")
    sw=request.args.get("sepal_w")
    pl=request.args.get("petal_l")
    pw=request.args.get("petal_w")
    parm=np.array([[sl,sw,pl,pw]])
    
    with graph.as_default(): # use the global graph
        predicted=model.predict_classes(parm)
        return names[predicted[0]]
    
if __name__=="__main__":
    app.run(host="0.0.0.0",port=5000)

# ビルトインWebサーバー起動

In [3]:
%run 05_whichiris/docroot/app.py

 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
